### Shannon-Hartley Theorem ~1940

$C = W \cdot \log_2(1+\frac{S}{N})$

C: achievable bit-rate

W: bandwith in Hz
S: Signal strength in W
N: Noise strength in W

SNR: Signal to Noise Ratio in dB

dB = $10 \cdot \log\frac{S}{N}$

dB is used as a *ratio* in telecommunications because line attenuation (or amplification) can simply be calculated by adding the dB ratios together (rather than multiplying the actual ratios)

Note: the noise characteristic is assumed to be "gaussian" (multitude of additive noise signals will lead to a gaussian distribution). This can be assumed for most of our basic understanding.

NB2: these calculations are upper-bound theoretical limits for bi-valent (two-level) data transmission over noisy channels.

An analog PSTN (old analogue telephone network) has a bandwidth of 3kHz and a typical attenuation of 20dB.

My current (lousy) WLAN connection has a bandwidth of 40MHz (channel bandwidth) an a signal strenght of -79dB, noise at 86dB, thus the SNR is 7dB (the difference) -- see how dB's are handy?

In [1]:
from math import pow, log2

def db(n):
    return pow(10, n/10)

def shannon_hartley_bitrate(w, snr):
    return w * log2(1+db(snr))

# an anaolg PSTN (back in the old days of steam engines and such)
W = 3E3
SNR_db = 20
print("Analogue PSTN bitrate={}".format(shannon_hartley_bitrate(W, SNR_db)))

# my current WLAN connection
W = 40E6
SNR_db = 6
print("Lousy WLAN bitrate={}".format(shannon_hartley_bitrate(W, SNR_db)))

Analogue PSTN bitrate=19974.634448255383
Lousy WLAN bitrate=92658247.18505038


(always calculate approximate values to exceedingly many digits :) )

#### Exercises

* Calculate the theoretical bitrate of you current WLAN connection (note: the channel-bandwith is (digitaly) divided in up- and down-stream, this calculates the total of both)

* Calculate the theoretical bitrate of an Cat-5 Twisted-Pair cable of 30m length. Use this [reference](http://www.cablek.com/technical-reference/cat-5---5e--6--6a---7--8-standards) (attenuation per 100m)

* the bitrate of the pioneer-10 probe

* the bitrate of an optical (fiber) channel

-----------

## Redundancy

To counter the errors induced by electromagnetic interference a certain amount of "redundancy" is
introduced in the bit-stream before transmission. The receiver is then able to determind the
error-free transmission (detection) or using more complex redundancy schemes to even correct some errors.

There goal is to keep this "redundancy" at a minimum while still achieve very low error-rates.

### Forward-Error-Correction FEC

The message to transmit is simply repeated three or more times, enabling the receiver to
determine the correct value of each bit by applying a majority evaluation.

This is a very crude scheme with a very bad channel-bandwidth (bitrate) efficiency.

Still sometimes used in *broadcast* communications (one-to-many transmission) where a single
receiver will not be able to request a retransmission. Although more modern schemes are used
for new developments.

### Parity-Bit

The goal of the parity-bit is to detect single bit errors in a bit-string (usually a byte for asynchronous serial transfers by RS-232)

The "parity" can be defined as. Like in all protocols the sender and receiver must
agree on the setting before a transmission can take place (either by negotiation
in more complex schemes or by simple setting it manually):

* odd: the parity-bit is set by the sender to achieve an odd number of "1" bits

* even: the parity-bit is set to make the number of "1" bits even

The receiver counts the "1" bits (including the parity bit) and determines if
the transmission was error-free by comparing the "evenness" or "oddness" of
bits. Actual implementation in software does not "count" bits but apply a
bitwise XOR operation accross all bits (see below).

This simple scheme can only detect single-bit errors -- there is no error correction possible.

### Hamming-Codes

Links:
* [phantastic video (part 1)](https://www.youtube.com/watch?v=b3NxrZOu_CE)

### Code-Tables with large Hamming-Distances

### Modern Codes

* Reed-Solomon
* Turbo-Codes

### Cyclic-Redundany-Check CRC

* Fixed-size polynom calculation
* Fair error-detection (CRC-32 undetected error rate (BER) about $2.3\cdot10^{-10}$)
* no error correction
* used universally, eg in Ethernet

Links:
* [MIT](http://web.mit.edu/16.682/www/leca.pdf)
* [IEEE RFC3385](https://www.ietf.org/rfc/rfc3385.txt)
* [IEEE 802.3](https://www.ieee802.org/3/bv/public/Jan_2015/perezaranda_3bv_2_0115.pdf)

In [9]:
### Parity-Bit

from functools import reduce

# this implementation is using strings (actually list of characters) -- not the most efficient way :)
def to_binary_byte(n):
    return "{0:08b}".format(n)

def parity(b):
    return str(reduce(lambda x,y: int(x)^int(y), list(b))) # in python "^" denotes the XOR operator

parity(to_binary_byte(15))

def add_byte_parity(b, par='odd'):
    p = parity(b)
    if par=='odd' and p=='0':
        p = '1'
    elif par=='even' and p=='1':
        p = '0'
    return b+p

# add_parity(to_binary_byte(7))

def check_byte_parity(b, par='odd'):
    byte = b[:-1]
    p_bit = b[-1]
    p_calc = parity(byte)
    return (par=='odd' and )
    

00000111


'000001111'

In [74]:
a=to_binary_byte(42)
print(a)
print(a[:-1])
print(a[-1])
print(parity(a))

00101010
0010101
0
1


### BER -- Bit-Error-Rate

#### BER-Generator


In [68]:
from random import random

def ber_induce(b, err_rate=1e-3):
    return "".join([ str(int(not int(bit))) if random() < err_rate else bit for bit in b ])

a = to_binary_byte(42) # [ to_binary_byte(n) for n in range(0,255) ]
b = ber_induce(a, 0.1)
a = "".join(a)

print("{}:\n{}\n{}".format("same" if a==b else "differ", a, b))

differ:
00101010
01101010
